<a href="https://colab.research.google.com/github/fvangool/Deep-Learning-Specialization-Coursera/blob/main/Stickers_iter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install autogluon
! pip install wbgapi
!pip install dask[dataframe]

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

datas = pd.read_csv('/content/train.csv')
datas.drop("id", axis=1, inplace=True)
df_clean = datas.dropna(subset=['num_sold'])
df_clean['num_sold'] = np.log1p(df_clean['num_sold'])
#y = df_clean['num_sold']
#df_clean.drop("num_sold", axis=1, inplace=True)
print(df_clean.shape)

#data = df_clean.copy()
target = 'num_sold'

#test_data = df_clean.drop(train_data.index)
#train_data = df_clean.sample(frac=0.8, random_state=42)

#split the data
## sipl temporal
# Ensure the 'date' column is in datetime format
#data['date'] = pd.to_datetime(data['date'])
#split the data
train_data = df_clean.sample(frac=0.8, random_state=42)
test_data = df_clean.drop(train_data.index)
print(train_data.columns)
year_column= "year"
country_column = "country"

# Filter data for the training period
#train_data = data[(data['date'] >= '2010-01-01') & (data['date'] <= '2015-12-31')]
#test_data = data[(data['date'] >= '2016-01-01') & (data['date'] <= '2016-12-31')]


<ipython-input-3-aa96bc86f99e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['num_sold'] = np.log1p(df_clean['num_sold'])


(221259, 5)
Index(['date', 'country', 'store', 'product', 'num_sold'], dtype='object')


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import wbgapi as wb

class GDPTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, country_col='country', year_col='year'):
        self.country_col = country_col
        self.year_col = year_col
        self.country_dict = {
            "Canada": "CAN",
            "Finland": "FIN",
            "Italy": "ITA",
            "Kenya": "KEN",
            "Norway": "NOR",
            "Singapore": "SGP"
        }
        self.gdp_data = None

    def _get_gdp_data(self):
        """Fetch GDP data for all countries and years once."""
        start_period = 2010
        end_period = 2020

        gdp_data = wb.data.DataFrame(
            'NY.GDP.PCAP.CD', #'NY.GDP.MKTP.CD',
            self.country_dict.values(),
            time=range(start_period, end_period + 1)
        )
        gdp_data = gdp_data.reset_index()
        gdp_data = gdp_data.melt(id_vars=['economy'], var_name='year', value_name='gdp')
        gdp_data['year'] = gdp_data['year'].str.extract('(\d+)').astype(int)
        return gdp_data

    def fit(self, X, y=None):
        if self.gdp_data is None:
            self.gdp_data = self._get_gdp_data()
        return self

    def transform(self, X):
        if self.gdp_data is None:
            raise RuntimeError("GDP data has not been fetched. Call fit before transform.")

        X_copy = X.copy()

        # Map country names to codes
        X_copy['country_code'] = X_copy[self.country_col].map(self.country_dict)

        # Merge with GDP data
        X_copy = X_copy.merge(
            self.gdp_data,
            left_on=['country_code', self.year_col],
            right_on=['economy', 'year'],
            how='left'
        )

        # Drop intermediate columns if needed
        X_copy.drop(columns=['country_code', 'economy'], inplace=True)

        # Handle missing GDP values if necessary
        if X_copy['gdp'].isnull().any():
            print("Warning: Missing GDP data for some entries.")

        return X_copy


In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np

class StickersFeatureTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.predictors = None

    def fit(self, X, y=None):
        return self

    def transform(self, X):
          X_copy = X.copy()

        # 1. Time Components
          X_copy['date'] = pd.to_datetime(X_copy['date'])
          #X_copy['day_of_week'] = X_copy['date'].dt.dayofweek
          #X_copy['day'] = X_copy['date'].dt.day
          X_copy['month'] = X_copy['date'].dt.month
          #X_copy['quarter'] = X_copy['date'].dt.quarter
          X_copy['year'] = X_copy['date'].dt.year
          #X_copy['day_of_year'] = X_copy['date'].dt.dayofyear
          #X_copy['week_of_year'] = X_copy['date'].dt.isocalendar().week
          #X_copy['is_weekend'] = X_copy['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)
          #X_copy['is_month_start'] = X_copy.date.dt.is_month_start.astype(int)
          #X_copy['is_month_end'] = X_copy.date.dt.is_month_end.astype(int)


        # 2. Country - Product - Store Interactions
          #X_copy['country_store_interaction'] = X_copy['country'] + '_' + X_copy['store']
          X_copy['country_product_interaction'] = X_copy['country'] + '_' + X_copy['product'] #G
          #X_copy['store_product_interaction'] = X_copy['store'] + '_' + X_copy['product']
          X_copy['country_store_product_interaction'] = X_copy['country'] + '_' + X_copy['store'] + '_' + X_copy['product'] #G
          #X_copy['country_quarter_interaction'] = X_copy['country'] + '_' + X_copy['quarter'].astype(str)
          #X_copy['store_quarter_interaction'] = X_copy['store'] + '_' + X_copy['quarter'].astype(str)
          #X_copy['product_quarter_interaction'] = X_copy['product'] + '_' + X_copy['quarter'].astype(str) # G
          #X_copy['country_product_quarter_interaction'] = X_copy['country'] + '_' + X_copy['product'] + '_' + X_copy['quarter'].astype(str)
          #X_copy['store_product_quarter_interaction'] = X_copy['store'] + '_' + X_copy['product'] + '_' + X_copy['quarter'].astype(str) #G
          #X_copy['country_store_product_quarter_interaction'] = X_copy['country'] + '_' + X_copy['store'] + '_' + X_copy['product'] + '_' + X_copy['quarter'].astype(str) #G
          #X_copy['country_month_interaction'] = X_copy['country'] + '_' + X_copy['month'].astype(str)
         # X_copy['store_month_interaction'] = X_copy['store'] + '_' + X_copy['month'].astype(str)
          #X_copy['product_month_interaction'] = X_copy['product'] + '_' + X_copy['month'].astype(str)
          #X_copy['country_product_month_interaction'] = X_copy['country'] + '_' + X_copy['product'] + '_' + X_copy['month'].astype(str)
          #X_copy['store_product_month_interaction'] = X_copy['store'] + '_' + X_copy['product'] + '_' + X_copy['month'].astype(str)
         # X_copy['country_store_product_month_interaction'] = X_copy['country'] + '_' + X_copy['store'] + '_' + X_copy['product'] + '_' + X_copy['month'].astype(str)
          #X_copy = X_copy.drop(['quarter'], axis=1)
          #X_copy = X_copy.drop(['date'], axis=1)
          #print("X_copy columns = ",X_copy.columns)


        # 3. Seasonal components

        # One-hot encode days of week (since weekly pattern is very strong)

          #for day in range(7):
            # X_copy[f'is_day_{day}'] = (X_copy['day_of_week'] == day).astype(int)
        # Create mask for products with strong yearly seasonality
          yearly_seasonal_products = ['Holographic Goose', 'Kerneler', 'Kerneler Dark Mode']
          X_copy['has_yearly_pattern'] = X_copy['product'].isin(yearly_seasonal_products).astype(int)
          #X_copy['season'] = pd.cut(
           # X_copy['date'].dt.month,
           # bins=[0, 3, 6, 9, 12],
            #labels=['Winter', 'Spring', 'Summer', 'Fall']
           #)
          #X_copy['season_relevant'] = np.where(
           # X_copy['has_yearly_pattern'] == 1,
            #X_copy['season'],
            #'no_yearly_pattern'
          #)
          X_copy['month_sin'] = np.sin(2 * np.pi * X_copy['month']/12)
          #X_copy['month_cos'] = np.cos(2 * np.pi * X_copy['month']/12)
          #X_copy['day_sin'] = np.sin(2 * np.pi * X_copy['day_of_week']/7)
          #X_copy['day_cos'] = np.cos(2 * np.pi * X_copy['day_of_week']/7)
          #X_copy['group'] = (X_copy['year'] - 2020) * 48 + X_copy['month'] * 4 + X_copy['day'] // 7

          #X_copy = X_copy.drop(columns=['date'])
        #transformed_train = transformed_train.drop(columns=[datetime_column])

          return X_copy

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoder=OneHotEncoder(sparse_output=False, handle_unknown='ignore')):
        self.encoder = encoder
        self.categorical_cols = None
        self.encoder_ = None

    def fit(self, X, y=None):
        self.categorical_cols = X.select_dtypes(include=['object', 'category']).columns
        self.encoder_ = self.encoder.fit(X[self.categorical_cols])
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.categorical_cols is not None:
            encoded_data = self.encoder_.transform(X_copy[self.categorical_cols])
            encoded_df = pd.DataFrame(
                encoded_data,
                columns=self.encoder_.get_feature_names_out(self.categorical_cols),
                index=X_copy.index
            )
            X_copy = X_copy.drop(self.categorical_cols, axis=1)
            X_copy = pd.concat([X_copy, encoded_df], axis=1)
        return X_copy

class NumericalEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, encoder=StandardScaler()):
        self.encoder = encoder
        self.numerical_cols = None
        self.encoder_ = None

    def fit(self, X, y=None):
        self.numerical_cols = X.select_dtypes(include=[np.number]).columns
        self.encoder_ = self.encoder.fit(X[self.numerical_cols])
        return self

    def transform(self, X):
        X_copy = X.copy()
        if self.numerical_cols is not None:
            encoded_data = self.encoder_.transform(X_copy[self.numerical_cols])
            encoded_df = pd.DataFrame(
                encoded_data,
                columns=self.numerical_cols,  # Use original numerical column names
                index=X_copy.index
            )
            X_copy = X_copy.drop(self.numerical_cols, axis=1)
            X_copy = pd.concat([X_copy, encoded_df], axis=1)
        return X_copy


class DebugTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        print("DebugTransformer Output:")
        print(X.head())
        return X

pipeline = Pipeline([
    ('feature_creation', StickersFeatureTransformer()),  # Creates ~30 new features
    ('debug_step', DebugTransformer()),  # Check the output after feature creation
    #('transformerGDP', GDPTransformer(country_col='country', year_col='year')), # Ensure 'year' exists
    #('encode_categorical', CategoricalEncoder(OneHotEncoder(sparse_output=False, handle_unknown='ignore'))),
    #('scale_numerical', NumericalEncoder(StandardScaler())),
])

transformed_train = pipeline.fit_transform(train_data)
transformed_test = pipeline.transform(test_data)
#transformed_train = train_data

DebugTransformer Output:
             date    country              store            product  num_sold  \
60922  2011-11-08  Singapore  Stickers for Less       Kaggle Tiers  7.304516   
10862  2010-05-01     Norway  Discount Stickers       Kaggle Tiers  6.923629   
47898  2011-06-17    Finland  Discount Stickers           Kerneler  5.894403   
214250 2016-07-08      Kenya  Stickers for Less  Holographic Goose  1.791759   
229502 2016-12-25     Canada  Discount Stickers       Kaggle Tiers  6.375025   

        month  year country_product_interaction  \
60922      11  2011      Singapore_Kaggle Tiers   
10862       5  2010         Norway_Kaggle Tiers   
47898       6  2011            Finland_Kerneler   
214250      7  2016     Kenya_Holographic Goose   
229502     12  2016         Canada_Kaggle Tiers   

                country_store_product_interaction  has_yearly_pattern  \
60922    Singapore_Stickers for Less_Kaggle Tiers                   0   
10862       Norway_Discount Stickers_Kagg

In [ ]:
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_absolute_percentage_error
from datetime import datetime

import pandas as pd




#set path for model save
#save_path = '/Users/metiomai/Documents/Kaggle/insurance/'
save_path = '/content/drive/My Drive/elm/'
#save_path = 'C:/Users/fvang/Downloads/playground-series-s4e12/model2/'
# Define target column and split into train/test
target = 'num_sold'
#split the data


#set runtime limit
time_limit=7200#21600



current_timestamp = datetime.now()
print(f"Start training: {current_timestamp}")
# Initialize and train AutoGluon
#hyperparameters={'CAT':{}, 'GBM':{}, 'RF':{},'KNN':{}}
try:
  predictor = TabularPredictor(label=target, path=save_path,
                             eval_metric='mean_absolute_percentage_error').fit(transformed_train,
                          verbosity=3,presets='best_quality',time_limit=time_limit)
except:
  current_timestamp = datetime.now()
  print(f"Execution interrupted at: {current_timestamp}")
current_timestamp = datetime.now()
print(f"Execution succesfully completed at: {current_timestamp}")

Verbosity: 3 (Detailed Logging)


Start training: 2025-01-18 07:35:00.464498


=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
GPU Count:          0
Memory Avail:       11.35 GB / 12.67 GB (89.5%)
Disk Space Avail:   48.42 GB / 100.00 GB (48.4%)
Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': True, 'num_bag_sets': 1, 'verbosity': 3}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': True,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_callbacks': False,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory

(_ray_fit pid=11316) [50]	valid_set's l2: 0.0318412	valid_set's mean_absolute_percentage_error: -0.0306634
(_ray_fit pid=11316) [150]	valid_set's l2: 0.00674027	valid_set's mean_absolute_percentage_error: -0.0121914 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(_ray_fit pid=11315) [200]	valid_set's l2: 0.00547007	valid_set's mean_absolute_percentage_error: -0.0111223 [repeated 3x across cluster]
(_ray_fit pid=11315) [350]	valid_set's l2: 0.00448556	valid_set's mean_absolute_percentage_error: -0.0100127 [repeated 7x across cluster]
(_ray_fit pid=11316) [600]	valid_set's l2: 0.00409966	valid_set's mean_absolute_percentage_error: -0.00949637 [repeated 6x across cluster]
(_ray_fit pid=11315) [600]	valid_set's l2: 0.0041121	valid_set's mean_absolute_percentage_error: -0.00959591 [repeated 5x acros

In [ ]:
#testy = pd.read_csv("/kaggle/input/playground-series-s4e12/test.csv") #set path to kaggle
testy = pd.read_csv('/content/test.csv')
#testy = pd.read.csv("C:/Users/fvang/Downloads/playground-series-s4e12/test.csv")
#testy = pd.read.csv('/kaggle/input/playground-series-s4e12/test.csv')
id = testy['id']
testy.drop("id", axis=1, inplace=True)

transformed_testy = testy

#transformed_testy = pipeline.transform(testy)

print("file loaded")
print("starting to predict")
y_testset_pred = predictor.predict(transformed_testy)
print("predicting completed")
print("inveresing the log")
y_testset_pred = np.expm1(y_testset_pred.values)
print("inversion completed")
num_sold = pd.DataFrame(y_testset_pred, columns=['num_sold'])
num_sold_round = np.around(num_sold['num_sold'])
submission = pd.concat([id, num_sold_round], axis=1)
print(submission.columns)
print(submission.head())
#submission.to_csv('submission2H.csv', index=False, sep=',')
submission.to_csv('/content/drive/My Drive/elm/submissionHope2.csv', index=False, sep=',')
print("submission file updated")

In [ ]:
performance = predictor.evaluate(transformed_test)
print(performance)

In [ ]:
performance_train = predictor.evaluate(transformed_train)
print(performance_train)

In [ ]:
predictor.leaderboard(transformed_test)